In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# 크롬 옵션 설정
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 드라이버 실행
driver = webdriver.Chrome(options=options)

# 웹페이지 열기
url = 'https://carinfo.knia.or.kr/lmxsrv/cnswc/icnyCnswcList.do'
driver.get(url)
time.sleep(3)

# HTML 파싱
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

results = []

companies = soup.select('div.con-box')
for company in companies:
    company_name = company.select_one('h4').text.strip()

    # 전화번호: "전화 :"가 포함된 li에서 텍스트 추출
    phone = ''
    for li in company.select('ul li'):
        if '전화' in li.text:
            phone = li.get_text(strip=True).replace('전화 :', '').strip()
            break

    # 버튼 URL 추출
    buttons = company.select('div.btn-box button')
    urls = []
    for btn in buttons:
        onclick = btn.get('onclick')
        if onclick:
            start = onclick.find("'") + 1
            end = onclick.rfind("'")
            extracted_url = onclick[start:end]
            urls.append(extracted_url)

    results.append({
        '회사명': company_name,
        '대표번호': phone,
        '상담센터': urls[0] if len(urls) > 0 else '',
        '민원창구': urls[1] if len(urls) > 1 else ''
    })

driver.quit()

# CSV 저장
df = pd.DataFrame(results)
df.to_csv('보험사_URL_전화번호.csv', index=False, encoding='utf-8-sig')
print("✅ '보험사_URL_전화번호.csv' 저장 완료!")


✅ '보험사_URL_전화번호.csv' 저장 완료!


In [3]:
%pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----------------------------- ---------- 1.6/2.1 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
from sqlalchemy import create_engine

# CSV 파일 경로
csv_file = '보험_할인_특약.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file)

# MySQL 접속 정보
username = 'SKN13'
password = '1111'
host = '127.0.0.1'
port = '3306'
database = 'car_insurance'
table_name = 'car_insurance'

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# 데이터프레임을 MySQL 테이블로 저장 (기존 테이블에 덮어쓰기: if_exists='replace')
df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

print("CSV 파일이 MySQL로 업로드되었습니다.")


CSV 파일이 MySQL로 업로드되었습니다.


In [7]:
import pandas as pd
from sqlalchemy import create_engine

# CSV 파일 경로
csv_file = '보험사_URL_전화번호.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file)

# MySQL 접속 정보
username = 'SKN13'
password = '1111'
host = '127.0.0.1'
port = '3306'
database = 'car_insurance'
table_name = 'car_number'

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# 데이터프레임을 MySQL 테이블로 저장 (기존 테이블에 덮어쓰기: if_exists='replace')
df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

print("CSV 파일이 MySQL로 업로드되었습니다.")


CSV 파일이 MySQL로 업로드되었습니다.


In [5]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.
